In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image
import os
import tensorflow as tf
import keras
from keras import layers
from keras import ops
import keras_tuner
from tensorflow.keras.applications.xception import preprocess_input

## Importing Data from csv files


In [6]:
images = pd.read_csv("CUB_200_2011/images.txt", sep=r'\s+', names=['image_id', 'image_name'], engine='python')
train_test_split = pd.read_csv("CUB_200_2011/train_test_split.txt", sep=r'\s+', names=['image_id', 'is_training_image'], engine='python')
classes =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/classes.txt", sep=r'\s+', names=['class_id', 'class_name'], engine='python')
image_class_labels =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/image_class_labels.txt", sep=r'\s+', names=['image_id', 'class_id'], engine='python')

In [11]:
print(images.head())
print(train_test_split.head())
print(image_class_labels.head())
print(classes.head())

   image_id                                         image_name
0         1  001.Black_footed_Albatross/Black_Footed_Albatr...
1         2  001.Black_footed_Albatross/Black_Footed_Albatr...
2         3  001.Black_footed_Albatross/Black_Footed_Albatr...
3         4  001.Black_footed_Albatross/Black_Footed_Albatr...
4         5  001.Black_footed_Albatross/Black_Footed_Albatr...
   image_id  is_training_image
0         1                  0
1         2                  1
2         3                  0
3         4                  1
4         5                  1
   image_id  class_id
0         1         1
1         2         1
2         3         1
3         4         1
4         5         1
   class_id                  class_name
0         1  001.Black_footed_Albatross
1         2        002.Laysan_Albatross
2         3         003.Sooty_Albatross
3         4       004.Groove_billed_Ani
4         5          005.Crested_Auklet


## Preprocessing

In [12]:
print(len(images))
print(len(train_test_split))
print(len(image_class_labels))
print(len(classes))

11788
11788
11788
200


In [18]:
# Merge dfs based on column names so we have one df with all the necessary info contained per each row
image_data = pd.merge(images,train_test_split, on='image_id')
image_data = pd.merge(image_data,image_class_labels, on='image_id')
image_data = pd.merge(image_data,classes, on='class_id')
print(image_data.to_string())
print(len(image_data))

       image_id                                                                         image_name  is_training_image  class_id                          class_name
0             1                      001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg                  0         1          001.Black_footed_Albatross
1             2                      001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg                  1         1          001.Black_footed_Albatross
2             3                      001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg                  0         1          001.Black_footed_Albatross
3             4                      001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg                  1         1          001.Black_footed_Albatross
4             5                      001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg                  1         1          001.Black_footed_Albatross
5             6 

In [20]:
# Split training and testing image data
training_image_data = image_data[image_data['is_training_image']==1]
testing_image_data = image_data[image_data['is_training_image']==0]

# Shuffle training data
training_image_data = training_image_data.sample(frac=1)

# Initiate empty lists for training and testing images
training_images = []
testing_images = []

# Add training and testing images to corresponding lists
for i in (training_image_data['image_name'].values):
    training_images.append(image.load_img('CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

for i in (testing_image_data['image_name'].values):
    testing_images.append(image.load_img('CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

# Extract class labels for training and testing images
training_class_label = np.array(training_image_data['class_id'].values)
testing_class_label = np.array(testing_image_data['class_id'].values)

In [24]:
# Convert list of images to NumPy array
training_images = np.array(training_images)
testing_images = np.array(testing_images)

# Apply preprocessing
preprocessed_training_images = preprocess_input(training_images)
preprocessed_testing_images = preprocess_input(testing_images)

In [18]:
# Fine-tuning xception model
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)

#globalaveragepooling turns it in to a vector
output = keras.layers.Dense(201, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [19]:
# Training with existing layers fixed
#we first want to freeze the previous layers except the last one
#want to make sure the ouput layer works first essentially then unfreeze previous layers and train again
#this allows us to optimize and ensure the rest are iptimized
#hard for the previous layers to optimize properly

dataset_size = len(training_images)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.05,
    decay_steps=10000,
    decay_rate=0.9)

from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss",
                                        mode="min",
                                        patience=3,
                                        restore_best_weights=True)


for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


history = model.fit(preprocessed_training_images[0], preprocessed_training_images[1], epochs=30, validation_split = 0.3, batch_size = 32, callbacks=[earlystopping]) #steps_per_epoch=dataset_size)#0.75//batch_size) #validation_data=valid_set, validation_steps = dataset_size*0.15//batch_size)
print()



Epoch 1/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1060s 8s/step - accuracy: 0.0833 - loss: 4.7469 - val_accuracy: 0.2724 - val_loss: 3.1069
Epoch 2/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1132s 9s/step - accuracy: 0.4003 - loss: 2.5663 - val_accuracy: 0.3813 - val_loss: 2.5085
Epoch 3/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1096s 8s/step - accuracy: 0.5799 - loss: 1.8309 - val_accuracy: 0.4230 - val_loss: 2.2974
Epoch 4/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1851s 14s/step - accuracy: 0.6619 - loss: 1.4580 - val_accuracy: 0.4364 - val_loss: 2.1967
Epoch 5/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1823s 14s/step - accuracy: 0.7144 - loss: 1.2354 - val_accuracy: 0.4719 - val_loss: 2.0485
Epoch 6/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1859s 14s/step - accuracy: 0.7772 - loss: 1.0322 - val_accuracy: 0.4725 - val_loss: 2.0082
Epoch 7/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 1824s 14s/step - accuracy: 0.8084 - loss: 0.9173 - val_accuracy: 0.4953 - val_loss: 1.9592
Epoch 8/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 2056s 16s/step - accuracy: 0.8272 - loss: 0.85

In [ ]:
   
# The model can be further trained with base layers unfrozen
for layer in base_model.layers:
  layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(preprocessed_training_images[0], preprocessed_training_images[1], epochs=30, validation_split = 0.3, callbacks=[earlystopping]) # steps_per_epoch=dataset_size*0.75//batch_size, validation_data=valid_set, validation_steps = dataset_size*0.15//batch_size)


Epoch 1/3
113/113 ━━━━━━━━━━━━━━━━━━━━ 2932s 26s/step - accuracy: 0.6231 - loss: 1.5186 - val_accuracy: 0.0000e+00 - val_loss: 12.7936
Epoch 2/3
113/113 ━━━━━━━━━━━━━━━━━━━━ 2781s 24s/step - accuracy: 0.8890 - loss: 0.4323 - val_accuracy: 0.0013 - val_loss: 11.1061
Epoch 3/3
113/113 ━━━━━━━━━━━━━━━━━━━━ 3250s 29s/step - accuracy: 0.9798 - loss: 0.1249 - val_accuracy: 0.0013 - val_loss: 10.6782


In [ ]:
model.evaluate(preprocessed_testing_images[0], preprocessed_testing_images[1])

182/182 ━━━━━━━━━━━━━━━━━━━━ 1003s 5s/step - accuracy: 0.6479 - loss: 1.9867


[4.9380340576171875, 0.42716604471206665]

In [ ]:
keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=None,
    shear_range=0.2,
    zoom_range=0.3,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=Preprocess(),
    validation_split=0.4,
)

'''img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
'''
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [20]:
len(preprocessed_testing_images[1])

5794

In [21]:
print(image_data.shape)

(11788, 9)
